In [24]:
# read csv file and convert to json
import pandas as pd
import json
import uuid

In [25]:
# users
df = pd.read_csv('users_rows.csv')
df.to_json('users_rows.json', orient='records')

user_ids = []

with open('users_rows.json') as f:
    json_data = json.load(f)
    for i in range(len(json_data)):
        user_id = str(uuid.uuid4())
        user_ids.append(user_id)
        json_data[i]["id"] = user_id
        json_data[i]["email"] = user_id + "@gmail.com"
        json_data[i]["ios_id"] = user_id.upper()
        json_data[i]["fcm_token"] = f"fcm_{user_id}"

with open('users_rows.json', 'w') as f:
    json.dump(json_data, f, indent=4)

In [26]:
df = pd.read_csv('models_rows.csv')
df.to_json('models_rows.json', orient='records')
# read json file and modify 
with open('models_rows.json') as f:
    json_data = json.load(f)
    for i in range(len(json_data)):
        user_id = user_ids[i%len(user_ids)]
        json_data[i]["user_id"] = user_id
        json_data[i]["id"] = str(uuid.uuid4())
        p = json_data[i]["parameters"]
        p = json.loads(p)
        del p["training_urls"]
        json_data[i]["parameters"] = json.dumps(p)
        a = json_data[i]["face_attributes"]
        a = json.loads(a)
        a = {"age": f["age"]}
        json_data[i]["face_attributes"] = json.dumps(a)

# write json file
with open('models_rows.json', 'w') as f:
    json.dump(json_data, f, indent=4)


In [33]:
# templates
df = pd.read_csv('templates_rows.csv')
df.to_json('templates_rows.json', orient='records')

with open('templates_rows.json') as f:
    json_data = json.load(f)
    for i in range(len(json_data)):
        json_data[i]["id"] = str(uuid.uuid4())
        json_data[i]["example_images"] = "[]"
        del json_data[i]["example_images_male"]
        del json_data[i]["example_images_female"]
        s = json_data[i]["settings"]
        s = json.loads(s)
        s = {"steps:": s["steps"]}
        json_data[i]["settings"] = json.dumps(s)

with open('templates_rows.json', 'w') as f:
    json.dump(json_data, f, indent=4)

# Convert to SQL

In [29]:
"""
convert json to .sql INSERT statements
Like:
INSERT INTO "public"."users" ("id", "email", "payment_status", "ios_id", "fcm_token", "credits", "finetune_access", "experiment") VALUES (
    '00000000-0000-0000-0000-000000000000',
    'test@gmail.com',
    'paid',
    'ABCD-1234-ABCD-1234',
    'acf5d3f4-5d3f-4acf-5d3f-4acf5d3f4acf',
    100,
    true,
    null
);

"""

import json
import uuid

with open('users_rows.json') as f_in:
    json_data = json.load(f_in)
    with open('users_rows.sql', 'w') as f_out:
        for i in range(len(json_data)):
            f_out.write("INSERT INTO \"public\".\"users\" (\"id\", \"email\", \"payment_status\", \"ios_id\", \"fcm_token\", \"credits\", \"finetune_access\", \"experiment\") VALUES (\n")
            f_out.write("\t\'" + json_data[i]["id"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["email"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["payment_status"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["ios_id"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["fcm_token"] + "\',\n")
            f_out.write("\t" + str(json_data[i]["credits"]) + ",\n")
            f_out.write("\t" + str(json_data[i]["finetune_access"]) + ",\n")
            f_out.write("\t" + (("'" + str(json_data[i]["experiment"]) + "'") if json_data[i]["experiment"] else "NULL" )+ "\n")
            f_out.write(");\n\n")
    
    

In [30]:
"""

CREATE TABLE "public"."templates" (
    "id" "uuid" DEFAULT "gen_random_uuid"() NOT NULL,
    "created_at" timestamp with time zone DEFAULT "now"(),
    "updated_at" timestamp with time zone DEFAULT "now"(),
    "name" "text" NOT NULL,
    "category" "text" NOT NULL,
    "description" "text" NOT NULL,
    "example_images" "jsonb" NOT NULL,
    "type" "text" NOT NULL,
    "settings" "jsonb" NOT NULL,
    "gender" "text" DEFAULT 'both'::"text",
    "is_bestof" boolean DEFAULT false NOT NULL,
    "example_images_male" "jsonb",
    "example_images_female" "jsonb"
);

"""

with open('templates_rows.json') as f_in:
    json_data = json.load(f_in)
    with open('templates_rows.sql', 'w') as f_out:
        for i in range(len(json_data)):
            f_out.write("INSERT INTO \"public\".\"templates\" (\"id\", \"created_at\", \"updated_at\", \"name\", \"category\", \"description\", \"example_images\", \"type\", \"settings\", \"gender\", \"is_bestof\") VALUES (\n")
            f_out.write("\t\'" + json_data[i]["id"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["created_at"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["updated_at"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["name"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["category"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["description"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["example_images"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["type"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["settings"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["gender"] + "\',\n")
            f_out.write("\t" + str(json_data[i]["is_bestof"]) + "\n")
            f_out.write(");\n\n")
            

In [31]:
"""

CREATE TABLE "public"."models" (
    "id" "uuid" DEFAULT "gen_random_uuid"() NOT NULL,
    "user_id" "uuid" NOT NULL,
    "training_images" "jsonb" NOT NULL,
    "created_at" timestamp with time zone DEFAULT "now"(),
    "updated_at" timestamp with time zone DEFAULT "now"(),
    "status" "text" NOT NULL,
    "estimated_time_to_completion" timestamp with time zone,
    "model_name" "text" NOT NULL,
    "gender" "text" DEFAULT 'both'::"text",
    "face_attributes" "jsonb"
);
"""

with open('models_rows.json') as f_in:
    json_data = json.load(f_in)
    with open('models_rows.sql', 'w') as f_out:
        for i in range(len(json_data)):
            f_out.write("INSERT INTO \"public\".\"models\" (\"id\", \"user_id\", \"training_images\", \"created_at\", \"updated_at\", \"status\", \"estimated_time_to_completion\", \"model_name\", \"gender\", \"face_attributes\") VALUES (\n")
            f_out.write("\t\'" + json_data[i]["id"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["user_id"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["training_images"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["created_at"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["updated_at"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["status"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["estimated_time_to_completion"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["model_name"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["gender"] + "\',\n")
            f_out.write("\t\'" + json_data[i]["face_attributes"] + "\'\n")
            f_out.write(");\n\n")